### request Vs urlib.request

- 데이터를 보낼때 requests는 딕셔너리 형태, urlib는 인코딩하여 바이너리 형태로 전송
- requests는 요청 메소드 (get,post)를 명시하지만 urlib는 데이터 여부에 따라 get과 post요청을 구분
- 없는 페이지 요청시 requests는 에러는 띄우지 않지만 urllib는 에러를 띄움

In [11]:
import requests
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup as bs
import re   # 데이터에 대한 정규형 표현
import pandas as pd

In [12]:
def news_parser(keyword,spage = 1, epage = 2):
    
    
    news_list=[]
    for page in range(int(spage), int(epage)+1):
        url = 'https://news.joins.com/Search/JoongangNews?page='+str(page)
        url = url +"&Keyword="+quote(keyword) #바이너리 코드 변경
        url = url + '&SortType=New&SearchCategoryType=JoongangNews'
    
        news_html = requests.get(url)
        if news_html.status_code != 200:
            print('url을 받아오지 못했습니다')
        news_soup = bs(news_html.text, 'html.parser')
    
    #방법 1 : select는 find_all과 같음
        li_text_tag = news_soup.select('div.bd>ul.list_default>li')
    
    #방법 2
#     ul_text_tag = news_soup.find('ul', class_="list_default")
#     li_text_tag = ul_text_tag.find_all('li')
    #print(li_text_tag)
    
        for li in li_text_tag:
            title = li.find('h2').get_text()
            new_url = li.find('a')['href']
            body= body_text(new_url)
            day = li.select('span.byline>em')[-1].get_text()

            news_list.append([title, day, body])

    df1 = pd.DataFrame(news_list, columns = ['제목', '날짜', '상세내용'])
    return df1




In [13]:
def body_text(url):
    html = urlopen(url)
    soup = bs(html,'html.parser')
    
    try: 
        body = soup.find('div', id='article_body').text
    except:
        body=""
        
    #숫자/영문/한글 모두 ""로 변환
    body = re.sub("[^0-9ㄱ-ㅣ가-힣a-zA-z]","",body)
    return body

In [14]:
keyword = input('조회 키워드 입력:')
spage = input('시작 페이지 번호:')
epage = input('종료 페이지 번호:')

df1 = news_parser(keyword, spage, epage)

조회 키워드 입력:올림픽
시작 페이지 번호:1
종료 페이지 번호:2


In [15]:
df1

,제목,날짜,상세내용
0,"대표팀 LG-상무와 평가전 추가 확정, 26일 출국",2021.07.21 13:52,2021도쿄올림픽야구대표팀이지난18일고척돔에서마스크를착용하고있다도쿄올림픽야구대표팀이...
1,개막 이틀 전에 극적으로 올림픽 티켓 얻은 유도 이성호,2021.07.21 13:43,국제유도연맹홈페이지캡처유도남자81급이성호한국마사회가2020도쿄올림픽개막을이틀앞두고극...
2,"아버지가 가지 못한 올림픽 길, '바람의 손자'가 뚫는다",2021.07.21 13:41,도쿄올림픽야구대표팀이정후[연합뉴스]바람의아들이종범51현LG2군타격코치은한국프로야구가...
3,'문 대통령 방일취소' 잘했다는 中 매체 “방일 포기 여론 부합”,2021.07.21 13:30,문재인대통령조바이든미국대통령스가요시히데일본총리[중앙포토]문재인대통령이도쿄올림픽기간에...
4,"조진웅 감독 데뷔작 북미영화제 초청 ""누구든 영웅 될 수 있죠""",2021.07.21 12:52,배우조진웅왼쪽이감독데뷔작으로해외영화제에초청됐다사진은2019년제24회부산국제영화제개막...
5,범 말고 곰 내려왔다…日올림픽 첫경기장 인근 출몰 '발칵',2021.07.21 12:11,일본공원에출몰한곰사진은기사와직접적관련이없습니다AFP연합뉴스도쿄올림픽소프트볼경기장인일...
6,올림픽 후원 삼성전자 '속앓이'…개막식엔 현지 실무진만 참석,2021.07.21 11:58,지난17일도쿄하루미지역올림픽선수촌입구에서보안요원들이검문검색을하고있다도쿄올림픽조직위원...
7,도쿄올림픽에서 42번째 우승에 도전하는 다니 알베스,2021.07.21 11:43,브라질상파울루에서뛰고있는다니알베스[로이터연합뉴스]메시보다더많이정상에오른사나이감독보다...
8,도쿄올림픽 개막 뒤 취소?…초유의 사태 가능성 커졌다,2021.07.21 11:33,도쿄올림픽이대회를치르는도중에중단되는초유의사태가벌어질수도있다도쿄올림픽개막을나흘앞둔19...
9,"오사카 나오미 수영복 차림 화보 논란…""인터뷰는 안 하면서""",2021.07.21 11:31,오사카나오미SNS여자테니스세계랭킹2위오사카나오미24일본선수가수영복차림으로스포츠잡지표...


In [ ]:
for df_url in df1['상세_url']:
                  
    html = urlopen(df_url)
    soup = bs(html, 'html.parser')

    title = soup.find('h1', class_='headline mg').text
    #print(title,"\n\n")
    body = soup.find('div', id='article_body').text
    #print(body)

In [ ]:
title_list=[]
body_list=[]

for df_url in df1['상세_url']:
                  
    html = urlopen(df_url)
    soup = bs(html, 'html.parser')

    title = soup.find('h1', class_='headline mg').text
    #print(title,"\n\n")
    try: 
        body = soup.find('div', id='article_body').text
    except:
        body=""
        
    #숫자/영문/한글 모두 ""로 변환
    #title_list.append(re.sub("[0-9ㄱ-ㅣ가-힣a-zA-z]","",title))
    title_list.append(re.sub("[^0-9ㄱ-ㅣ가-힣a-zA-z]","",title))
    #print(body)

In [ ]:
txt = '''
안녕하세요~~
반갑습니다ㅋㅋㅋㅋ
body, html, H1, 1234'''

In [ ]:
print(re.sub('[a-zㄱ-ㅣ]',"",txt)) #제거
print(re.sub('[^a-z가-힣]',"",txt)) #남김